In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_tasks = pd.read_csv('./data/mysql_table_tasks.csv')
data_inc = pd.read_csv('./data/mysql_table_incidents.csv')
data_inc_types = pd.read_csv('./data/mysql_table_incident_types.csv')

data_tasks = pd.read_csv('./data/mysql_table_tasks.csv')
data_tasks_stages = pd.read_csv('./data/mysql_table_task_stages.csv')
data_tasks_transitions = pd.read_csv('./data/mysql_table_task_transitions.csv')
data_tasks_user = pd.read_csv('./data/mysql_table_task_user.csv')

In [ ]:
data_query_inc = data_inc[(data_inc['account_id'] == 2)]    # | (data_inc['account_id'] == 2)
#берем только закрытые инциденты
data_query_inc_closed = data_query_inc[(data_query_inc['closed_at'].isna() == False)]

In [ ]:
data_query_merged_title = data_query_inc.merge(data_inc_types, left_on='incident_type_id', right_on='id')

# Нужно сделать срез по подразеделниям и группировку по объектам и в разрезе подразеделений оценить каждый объект на `data_query_merged_title['title'].value_counts()` + failure type для этих задач тоже value_counts()

In [ ]:
data_division = pd.read_csv('./data/mysql_table_divisions.csv')
data_division[data_division['account_id'] == 2]

In [ ]:
data_task_user = pd.read_csv('./data/mysql_table_task_user.csv')
# data_task_to_divisions = pd.read_csv('./data/mysql_table_users.csv')

data_users = pd.read_csv('./data/mysql_table_users.csv')
join_object = pd.read_csv('./data/mysql_table_service_object_task.csv')
data_objects = pd.read_csv('./data/mysql_table_service_objects.csv')

In [ ]:
data_users_id = data_tasks[data_tasks['account_id'] == 1]
data_task_users_id_merged = data_tasks.merge(data_task_user, left_on='id', right_on='task_id')
data_task_users_merged = data_task_users_id_merged.merge(data_users, left_on='user_id', right_on='id')

In [ ]:
data_task_divisions = data_task_users_merged.merge(data_division, left_on='division_id', right_on='id')

In [ ]:
data_task_divisions.columns

In [ ]:
data_task_join_div_obj = data_task_divisions.merge(join_object, left_on='id_x', right_on='task_id')

In [ ]:
data_task_join_div_obj

In [ ]:
data_task_join_div_obj = data_task_join_div_obj[data_task_join_div_obj['taskable_type'] == 'App\Models\ServiceDesk\Incident']
data_task_join_div_obj.columns

In [ ]:
data_div_obj_finally = data_task_join_div_obj.merge(data_objects, left_on='service_object_id', right_on='id')

In [ ]:
data_div_obj_finally.columns

In [ ]:
data_clear_divobj = data_div_obj_finally.drop(['assigner_id', 'assignable_type', 'assignable_id', 'description', 'deadline_at', 'deleted_at_x', 'created_at_x', 'updated_at_x', 'name','email', 'phone', 'settings', 'email_verified_at', 'password', 'uses_personal_transport', 'remember_token', 'deleted_at_y', 'created_at_y', 'updated_at_y', 'is_supervisor', 'location', 'deleted_at_x', 'created_at_x', 'updated_at_x', 'account_id_y', 'is_demounted', 'deleted_at_y', 'created_at_y', 'updated_at_y'], axis=1)

In [ ]:
# data_clear_divobj.drop(['id_x'], axis=1, inplace=True)
# data_clear_divobj.drop(['account_id_x'], axis=1, inplace=True)

In [ ]:
data_clear_divobj = data_clear_divobj.merge(data_query_inc, left_on='taskable_id', right_on='id')
data_clear_divobj = data_clear_divobj.merge(data_inc_types, left_on='incident_type_id', right_on='id')

In [ ]:
data_clear_divobj

In [ ]:
data_clear_divobj.columns

In [ ]:
data_clear_divobj['title_x'].unique()

In [ ]:
data_clear_divobj['title']

### - Убрать полное отсутсвтие данных чтобы было адекват значение инцидентов

### - Указать числа для каждого инцидента

### -  анонимизация

In [ ]:
data_pie = data_clear_divobj[data_clear_divobj['title_x'] == 'Голубкин']['title'].value_counts()

In [ ]:
def autopct_format(values):
    def my_format(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{v:d}'.format(v=val)
    return my_format

data_to_print = {}
for idx, i in enumerate(data_clear_divobj.title_x.unique()):
    plt.figure(figsize=(12, 7))
    plt.title(f'Частота повторений инцидента у Пользователь № {idx + 1}')
    data_pie = pd.DataFrame(data_clear_divobj[data_clear_divobj['title_x'] == i]['title'].value_counts())
    data_pie = data_pie[data_pie.values < (sum(data_pie.values) / 2)]
    data_to_print[idx] = data_pie[data_pie.values >= (sum(data_pie.values) / 2)]
    pie_shares = list(data_pie.to_dict()['title'].values())
    positions = list(data_pie.to_dict()['title'].keys())
    plt.pie(pie_shares, labels=positions, autopct=autopct_format(pie_shares))
    plt.savefig(f'./results/pie_top_user{idx}.jpg', dpi=300)

In [ ]:
data_print_all_repeat_inc = []

plt.figure(figsize=(12, 7))
plt.title('Частота повторений инцидентов у Account_id = 1')
data_barh = pd.DataFrame(data_query_merged_title['title'].value_counts())
data_print_all_repeat_inc.append(data_barh[data_barh.values >= sum(data_barh.values) / 2])
data_to_draw = data_barh[data_barh.values < sum(data_barh.values) / 2].copy()

pie_shares = list(data_to_draw.to_dict()['title'].values())[:5]
positions = list(data_to_draw.to_dict()['title'].keys())[:5]
plt.pie(pie_shares, labels=positions, autopct=autopct_format(pie_shares))
plt.savefig(f'./results/all_repeat_inc.jpg', dpi=300)

In [ ]:
#Для Никиты!!!!!!!!!!!!
data_print_all_repeat_inc

# Средневзвешенное значение повторных инцидентов

In [ ]:
num_all_inc = int(sum(data_barh.values))
doles = [round(float(i / num_all_inc), 4) for i in data_to_draw.values]
num_chst = list(data_to_draw.values.ravel())

In [ ]:
sums = 0
for i in range(len(num_chst)):
    sums += num_chst[i] * doles[i]

aver_chast_weight = round(sums / sum(num_chst), 4)

In [ ]:
aver_chast_weight

# Комплексная оценка aver_chast_weight = среднее взвешенное значения повторяющихся инцидентов
all_repeat_inc - график общей повторяемости инцидентов без учета глоабльного инцидента data_print_all_repeat_inc

pie_top_user{idx}.jpg - n пирогов по каждому пользователю